In [ ]:
import requests
from conllu import parse
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, AutoModelForMaskedLM, AutoModel
from transformers import GPT2Tokenizer, GPT2Model
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch.nn.functional as F

# Attention Head Probing

In [ ]:
urls = {
    "dev": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-BOUN/refs/heads/master/tr_boun-ud-dev.conllu",
    "test": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-BOUN/refs/heads/master/tr_boun-ud-test.conllu",
    "train": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-BOUN/refs/heads/master/tr_boun-ud-train.conllu",
    "dev2": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Atis/refs/heads/master/tr_atis-ud-dev.conllu",
    "test2": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Atis/refs/heads/master/tr_atis-ud-test.conllu",
    "train2": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Atis/refs/heads/master/tr_atis-ud-train.conllu",
    "test3": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-GB/refs/heads/master/tr_gb-ud-test.conllu",
    "dev4": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Kenet/refs/heads/master/tr_kenet-ud-dev.conllu",
    "test4": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Kenet/refs/heads/master/tr_kenet-ud-test.conllu",
    "train4": "https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-Kenet/refs/heads/master/tr_kenet-ud-train.conllu"
}

def fetch_and_parse(url):
    response = requests.get(url)
    response.raise_for_status()
    return parse(response.text)

filtered = []
for name, url in urls.items():
    sentences = fetch_and_parse(url)
    for sent in sentences:
        nsubj_tokens = [tok for tok in sent if tok["deprel"] == "nsubj"]
        root_tokens = [tok for tok in sent if tok["deprel"] == "root"]
        if nsubj_tokens and root_tokens:
            filtered.append({
                "sentence": " ".join(tok["form"] for tok in sent),
                "nsubj": [tok["form"] for tok in nsubj_tokens],
                "root": [tok["form"] for tok in root_tokens],
                "split": name
            })


df = pd.DataFrame(filtered)
df.to_csv("nsubj_root_sentences_tr.csv", index=False)


In [ ]:
model_name = "dbmdz/bert-base-turkish-128k-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_attentions=True)
model.eval()

df = pd.read_csv("nsubj_root_sentences_tr.csv")

results = []

for i, row in df.iterrows():
    sentence = row["sentence"]
    nsubj = row["nsubj"].strip("[]'").split(",")[0].strip()
    root = row["root"].strip("[]'").split(",")[0].strip()

    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    try:
        nsubj_idx = tokens.index(nsubj)
        root_idx = tokens.index(root)
    except ValueError:
        continue

    if root_idx < nsubj_idx:
        continue

    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)

    attentions = outputs.attentions
    attn_tensor = torch.stack(attentions).squeeze(1)
    attention_scores = attn_tensor[:, :, root_idx, nsubj_idx]

    flat_idx = torch.argmax(attention_scores)
    best_layer = flat_idx.item() // attention_scores.shape[1]
    best_head = flat_idx.item() % attention_scores.shape[1]
    max_value = attention_scores[best_layer, best_head].item()

    results.append({
        "sentence": sentence,
        "nsubj": nsubj,
        "root": root,
        "nsubj_idx": nsubj_idx,
        "root_idx": root_idx,
        "best_layer": best_layer,
        "best_head": best_head,
        "attention_value": max_value
    })

pd.DataFrame(results).to_csv("turkish_bert_attention_nsubj_root.csv", index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/740M [00:00<?, ?B/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
model_name = "redrussianarmy/gpt2-turkish-cased"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  
model = GPT2Model.from_pretrained(model_name, output_attentions=True)
model.eval()

df = pd.read_csv("nsubj_root_sentences_tr.csv")

results = []

for i, row in df.iterrows():
    sentence = row["sentence"]
    nsubj = row["nsubj"].strip("[]'").split(",")[0].strip()
    root = row["root"].strip("[]'").split(",")[0].strip()

    inputs = tokenizer(sentence, return_tensors="pt", padding=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    try:
        root_idxs = [i for i, tok in enumerate(tokens) if root in tok]
        nsubj_idxs = [i for i, tok in enumerate(tokens) if nsubj in tok]
        if not root_idxs or not nsubj_idxs:
            continue
        root_idx = root_idxs[0]
        nsubj_idx = nsubj_idxs[0]
        if nsubj_idx >= root_idx:
            continue  
    except Exception:
        continue

    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)

    attentions = outputs.attentions 
    attn_tensor = torch.stack(attentions).squeeze(1) 

    attention_scores = attn_tensor[:, :, root_idx, nsubj_idx]  

    flat_idx = torch.argmax(attention_scores)
    best_layer = flat_idx.item() // attention_scores.shape[1]
    best_head = flat_idx.item() % attention_scores.shape[1]
    max_value = attention_scores[best_layer, best_head].item()

    results.append({
        "sentence": sentence,
        "nsubj": nsubj,
        "root": root,
        "root_idx": root_idx,
        "nsubj_idx": nsubj_idx,
        "best_layer": best_layer,
        "best_head": best_head,
        "attention_value": max_value
    })

pd.DataFrame(results).to_csv("gpt2_turkish_attention_nsubj_root.csv", index=False)


tokenizer_config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/594k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
def summarize_attention_weight(csv_path):
    df = pd.read_csv(csv_path)
    total_sentences = len(df)

    weighted_scores = (
        df.groupby(["best_layer", "best_head"])["attention_value"]
        .sum()
        .reset_index()
        .rename(columns={"attention_value": "total_weighted_attention"})
    )

    frequency = (
        df.groupby(["best_layer", "best_head"])
        .size()
        .reset_index(name="count")
    )

    summary = weighted_scores.merge(frequency, on=["best_layer", "best_head"])
    summary["percentage"] = 100 * summary["count"] / total_sentences

    top5 = summary.sort_values("total_weighted_attention", ascending=False).head(5)
    return top5


def summarize_attention_frequency(csv_path):
    df = pd.read_csv(csv_path)
    total = len(df)

    freq_table = (
        df.groupby(["best_layer", "best_head"])
        .size()
        .reset_index(name="count")
        .sort_values("count", ascending=False)
    )
    freq_table["percentage"] = 100 * freq_table["count"] / total

    return freq_table.head(5)

def get_top_layer_head(df, myrow="total_weighted_attention"):
    top_row = df.loc[df[myrow].idxmax()]
    LAYER = int(top_row["best_layer"])
    HEAD = int(top_row["best_head"])
    return LAYER, HEAD


In [6]:
print(summarize_attention_weight("gpt2_turkish_attention_nsubj_root.csv"))
print(summarize_attention_frequency("gpt2_turkish_attention_nsubj_root.csv"))
GPT2_LAYER, GPT2_HEAD = get_top_layer_head(summarize_attention_weight("gpt2_turkish_attention_nsubj_root.csv"))

    best_layer  best_head  total_weighted_attention  count  percentage
23           4          0                450.520395    451   26.591981
39           5          6                241.189950    245   14.445755
41           5          8                 96.133639     97    5.719340
61           7          8                 82.365050    111    6.544811
24           4          1                 75.100219     76    4.481132
    best_layer  best_head  count  percentage
23           4          0    451   26.591981
39           5          6    245   14.445755
61           7          8    111    6.544811
41           5          8     97    5.719340
68           8          3     86    5.070755


In [7]:
print(summarize_attention_weight("turkish_bert_attention_nsubj_root.csv"))
print(summarize_attention_frequency("turkish_bert_attention_nsubj_root.csv"))
BERT_LAYER, BERT_HEAD = get_top_layer_head(summarize_attention_frequency("turkish_bert_attention_nsubj_root.csv"), myrow="percentage")

    best_layer  best_head  total_weighted_attention  count  percentage
33           3          0               1734.309274   1736   17.157541
47           4          9               1200.819856   1202   11.879818
41           4          0                604.793325   1061   10.486262
74           7          8                349.915513    657    6.493378
56           5         11                320.874844    749    7.402649
    best_layer  best_head  count  percentage
33           3          0   1736   17.157541
47           4          9   1202   11.879818
41           4          0   1061   10.486262
56           5         11    749    7.402649
70           7          3    715    7.066614


# Attention

## GPT

In [ ]:
model_name = "ytu-ce-cosmos/turkish-gpt2"

config = AutoConfig.from_pretrained(model_name)
config.output_attentions = True

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, config=config)
model.eval()

df = pd.read_csv("syn_llm.csv", header=None,
    names=["language", "item", "syncretic", "sentence", "condition", "head_num", "attr_num", "verb_num"])
df = df[df["language"] == "turkish"]

def find_token_span(word, tokens):
    word = word.lower().strip(".,’'")
    word_pointer = 0
    indices = []
    buffer = ""

    for i, tok in enumerate(tokens):
        tok_str = tokenizer.convert_tokens_to_string([tok]).strip().lower()
        # Fix corrupted chars
        tok_str = (tok_str
                   .replace("â", "ı")
                   .replace("Ã§", "ç")
                   .replace("Å", "ş")
                   .replace("Ä±", "ı")
                   .replace("Ġ", "")
                   .replace("##", ""))
        buffer += tok_str
        indices.append(i)

        if buffer.startswith(word):
            if len(buffer) >= len(word):
                return indices
        elif not word.startswith(buffer):
            buffer = ""
            indices = []
    return []

def extract_attention_from_df(df, layer=GPT2_LAYER, head=GPT2_HEAD):
    rows = []

    for _, row in df.iterrows():
        sentence = row["sentence"]
        words = sentence.strip(".").split()
        if len(words) < 3:
            print(f"[SKIP] Too short: {sentence}")
            continue

        attractor = words[0]
        head_word = words[1]
        verb = words[-1]

        # Tokenize
        inputs = tokenizer(sentence, return_tensors="pt")
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

        attractor_span = find_token_span(attractor, tokens)
        head_span = find_token_span(head_word, tokens)
        verb_span = find_token_span(verb, tokens)

        if not attractor_span or not head_span or not verb_span:
            print(f"[SKIP] No span found in: {sentence}")
            continue

        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
            attn = outputs.attentions[layer][0, head]  # [seq, seq]

        attn_to_attr = attn[verb_span][:, attractor_span].sum().item()
        attn_to_head = attn[verb_span][:, head_span].sum().item()

        rows.append({
            "item": row["item"],
            "sentence": sentence,
            "condition": row["condition"],
            "head_num": row["head_num"],
            "attr_num": row["attr_num"],
            "verb_num": row["verb_num"],
            "attention_to_attractor": attn_to_attr,
            "attention_to_head": attn_to_head,
            "attractor_tokens": [tokens[i] for i in attractor_span],
            "head_tokens": [tokens[i] for i in head_span],
            "verb_tokens": [tokens[i] for i in verb_span],
        })

    return pd.DataFrame(rows)

turkish_gpt2_df = extract_attention_from_df(df)
print(turkish_gpt2_df[["sentence", "attention_to_head", "attention_to_attractor"]])

turkish_gpt2_df.to_csv("turkish_gpt2_attention_output.csv", index=False)


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[SKIP] No span found in: Hastaların cerrahı bile bile geç kaldılar
[SKIP] No span found in: Hastaların cerrahı bile bile geç kaldı
[SKIP] No span found in: Hastanın cerrahı bile bile geç kaldılar
[SKIP] No span found in: Hastanın cerrahı bile bile geç kaldı
                                              sentence  attention_to_head  \
0     yöneticilerin aşçısı mutfakta sürekli zıpladılar           0.010225   
1        yöneticilerin aşçısı mutfakta sürekli zıpladı           0.009859   
2       yöneticinin aşçısı mutfakta sürekli zıpladılar           0.009485   
3          yöneticinin aşçısı mutfakta sürekli zıpladı           0.009209   
4        öğrencilerin ablası sınıfta birden bayıldılar           0.029803   
..                                                 ...                ...   
303  Politikacının tercümanı toplantıdan sonra bird...           0.004431   
304  Mühendislerin işvereni yabancı bir ülkede kayb...           0.004966   
305  Mühendislerin işvereni yabancı bir ülkede ka

## BERT

In [ ]:
model_name = "dbmdz/bert-base-turkish-cased"
config = AutoConfig.from_pretrained(model_name)
config.output_attentions = True

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, config=config)
model.eval()

df = pd.read_csv("syn_llm.csv", header=None,
    names=["language", "item", "syncretic", "sentence", "condition", "head_num", "attr_num", "verb_num"])
df = df[df["language"] == "turkish"]

def get_token_span(word, tokens):
    word = word.lower().strip(".,’'")
    span = []
    pointer = 0
    for i, token in enumerate(tokens):
        clean = token.replace("##", "").lower().lstrip("▁")
        if word[pointer:].startswith(clean):
            span.append(i)
            pointer += len(clean)
        if pointer >= len(word):
            break
    return span

def extract_bert_attention_from_df(df, layer=BERT_LAYER, attn_head=BERT_HEAD):
    rows = []

    for _, row in df.iterrows():
        sentence = row["sentence"]
        words = sentence.strip(".").split()
        if len(words) < 3:
            print(f"[SKIP] Too short: {sentence}")
            continue

        attractor = words[0]
        head = words[1]
        verb = words[-1]

        inputs = tokenizer(sentence, return_tensors="pt")
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

        att_span = get_token_span(attractor, tokens)
        head_span = get_token_span(head, tokens)
        verb_span = get_token_span(verb, tokens)

        if not att_span or not head_span or not verb_span:
            print(f"[SKIP] No span found in: {sentence}")
            continue

        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
            attentions = outputs.attentions 

        attn_matrix = attentions[layer][0, attn_head]  

        attn_to_att = attn_matrix[verb_span][:, att_span].sum().item()
        attn_to_head = attn_matrix[verb_span][:, head_span].sum().item()

        rows.append({
            "item": row["item"],
            "sentence": sentence,
            "condition": row["condition"],
            "head_num": row["head_num"],
            "attr_num": row["attr_num"],
            "verb_num": row["verb_num"],
            "attention_to_attractor": attn_to_att,
            "attention_to_head": attn_to_head,
            "attractor_tokens": [tokens[i] for i in att_span],
            "head_tokens": [tokens[i] for i in head_span],
            "verb_tokens": [tokens[i] for i in verb_span],
        })

    return pd.DataFrame(rows)

turkish_bert_df = extract_bert_attention_from_df(df)
print(turkish_bert_df[["sentence", "attention_to_attractor", "attention_to_head"]])

turkish_bert_df.to_csv("turkish_bert_attention_verb_to_nouns.csv", index=False)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

                                              sentence  \
0     yöneticilerin aşçısı mutfakta sürekli zıpladılar   
1        yöneticilerin aşçısı mutfakta sürekli zıpladı   
2       yöneticinin aşçısı mutfakta sürekli zıpladılar   
3          yöneticinin aşçısı mutfakta sürekli zıpladı   
4        öğrencilerin ablası sınıfta birden bayıldılar   
..                                                 ...   
307  Politikacının tercümanı toplantıdan sonra bird...   
308  Mühendislerin işvereni yabancı bir ülkede kayb...   
309  Mühendislerin işvereni yabancı bir ülkede kayb...   
310  Mühendisin işvereni yabancı bir ülkede kaybold...   
311    Mühendisin işvereni yabancı bir ülkede kayboldu   

     attention_to_attractor  attention_to_head  
0                  0.035185           0.053760  
1                  0.050856           0.049862  
2                  0.139017           0.049499  
3                  0.147774           0.049087  
4                  0.054836           0.126764  
..       

In [ ]:
bert_df = turkish_bert_df[["item", "sentence", "condition", "head_num", "attr_num", "verb_num", "attention_to_attractor", "attention_to_head"]].copy()
bert_df["model"] = "bert"

gpt2_df_clean = turkish_gpt2_df[["item", "sentence", "condition", "head_num", "attr_num", "verb_num", "attention_to_attractor", "attention_to_head"]].copy()
gpt2_df_clean["model"] = "gpt2"

combined_df = pd.concat([bert_df, gpt2_df_clean], ignore_index=True)

print(combined_df)
combined_df.to_csv("combined_bert_gpt2_attention_turkish.csv", index=False)


    item                                           sentence    condition  \
0      1   yöneticilerin aşçısı mutfakta sürekli zıpladılar  condition_a   
1      1      yöneticilerin aşçısı mutfakta sürekli zıpladı  condition_b   
2      1     yöneticinin aşçısı mutfakta sürekli zıpladılar  condition_c   
3      1        yöneticinin aşçısı mutfakta sürekli zıpladı  condition_d   
4      2      öğrencilerin ablası sınıfta birden bayıldılar  condition_a   
..   ...                                                ...          ...   
615   37  Politikacının tercümanı toplantıdan sonra bird...  condition_d   
616   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_a   
617   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_b   
618   38  Mühendisin işvereni yabancı bir ülkede kaybold...  condition_c   
619   38    Mühendisin işvereni yabancı bir ülkede kayboldu  condition_d   

    head_num attr_num verb_num  attention_to_attractor  attention_to_head  \
0         

# Surprisal

## BERT

In [ ]:
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

df = pd.read_csv("syn_llm.csv", header=None,
    names=["language", "item", "syncretic", "sentence", "condition", "head_num", "attr_num", "verb_num"])
df = df[df["language"] == "turkish"]

results = []

for _, row in df.iterrows():
    sentence = row["sentence"]
    words = sentence.strip(".").split()
    if len(words) < 3:
        print(f"[SKIP] Too short: {sentence}")
        continue

    verb = words[-1]
    masked_words = words[:-1] + ["[MASK]"]
    masked_sent = " ".join(masked_words)

    try:
        inputs = tokenizer(masked_sent, return_tensors="pt")
        mask_idx = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1].item()

        with torch.no_grad():
            logits = model(**inputs).logits  # [batch, seq, vocab]

        probs = F.softmax(logits, dim=-1)
        verb_id = tokenizer.convert_tokens_to_ids(verb)
        prob = probs[0, mask_idx, verb_id].item()

        surprisal = -torch.log2(torch.tensor(prob)).item()

        results.append({
            "item": row["item"],
            "sentence": sentence,
            "condition": row["condition"],
            "head_num": row["head_num"],
            "attr_num": row["attr_num"],
            "verb_num": row["verb_num"],
            "verb": verb,
            "surprisal": surprisal,
            "model": "bert"
        })

    except Exception as e:
        print(f"[SKIP] {sentence} — {e}")
        continue

bert_surprisal_df = pd.DataFrame(results)
print(bert_surprisal_df)

bert_surprisal_df.to_csv("bert_turkish_verb_surprisal.csv", index=False)

    item                                           sentence    condition  \
0      1   yöneticilerin aşçısı mutfakta sürekli zıpladılar  condition_a   
1      1      yöneticilerin aşçısı mutfakta sürekli zıpladı  condition_b   
2      1     yöneticinin aşçısı mutfakta sürekli zıpladılar  condition_c   
3      1        yöneticinin aşçısı mutfakta sürekli zıpladı  condition_d   
4      2      öğrencilerin ablası sınıfta birden bayıldılar  condition_a   
..   ...                                                ...          ...   
307   37  Politikacının tercümanı toplantıdan sonra bird...  condition_d   
308   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_a   
309   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_b   
310   38  Mühendisin işvereni yabancı bir ülkede kaybold...  condition_c   
311   38    Mühendisin işvereni yabancı bir ülkede kayboldu  condition_d   

    head_num attr_num verb_num         verb  surprisal model  
0         sg       pl   

## GPT

In [ ]:
model_name = "ytu-ce-cosmos/turkish-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

df = pd.read_csv("syn_llm.csv", header=None,
    names=["language", "item", "syncretic", "sentence", "condition", "head_num", "attr_num", "verb_num"])
df = df[df["language"] == "turkish"]

results = []

for _, row in df.iterrows():
    sentence = row["sentence"]
    try:
        # Tokenize
        inputs = tokenizer(sentence, return_tensors="pt")
        input_ids = inputs["input_ids"]

        with torch.no_grad():
            logits = model(**inputs).logits 

        probs = F.softmax(logits, dim=-1)
        next_token_probs = probs[0, :-1, :]
        true_next_tokens = input_ids[0, 1:]

        surprisals = -torch.log2(next_token_probs[torch.arange(len(true_next_tokens)), true_next_tokens])

        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

        for tok, s in reversed(list(zip(tokens[1:], surprisals.tolist()))):
            clean = tok.replace("Ġ", "").replace("▁", "").strip().lower()
            if clean and clean not in [".", ",", ";", "!", "?"]:
                results.append({
                    "item": row["item"],
                    "sentence": sentence,
                    "condition": row["condition"],
                    "head_num": row["head_num"],
                    "attr_num": row["attr_num"],
                    "verb_num": row["verb_num"],
                    "token": tok,
                    "surprisal": s,
                    "model": "gpt2"
                })
                break
        else:
            print(f"[SKIP] No content token found: {sentence}")

    except Exception as e:
        print(f"[SKIP] {sentence} — {e}")
        continue

gpt2_surprisal_df = pd.DataFrame(results)
print(gpt2_surprisal_df)

gpt2_surprisal_df.to_csv("gpt2_turkish_verb_surprisal.csv", index=False)


    item                                           sentence    condition  \
0      1   yöneticilerin aşçısı mutfakta sürekli zıpladılar  condition_a   
1      1      yöneticilerin aşçısı mutfakta sürekli zıpladı  condition_b   
2      1     yöneticinin aşçısı mutfakta sürekli zıpladılar  condition_c   
3      1        yöneticinin aşçısı mutfakta sürekli zıpladı  condition_d   
4      2      öğrencilerin ablası sınıfta birden bayıldılar  condition_a   
..   ...                                                ...          ...   
307   37  Politikacının tercümanı toplantıdan sonra bird...  condition_d   
308   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_a   
309   38  Mühendislerin işvereni yabancı bir ülkede kayb...  condition_b   
310   38  Mühendisin işvereni yabancı bir ülkede kaybold...  condition_c   
311   38    Mühendisin işvereni yabancı bir ülkede kayboldu  condition_d   

    head_num attr_num verb_num      token  surprisal model  
0         sg       pl     

In [18]:
import numpy as np

bert_surprisal_df["surprisal"] = bert_surprisal_df["surprisal"] / np.log(2)
gpt2_surprisal_df["surprisal"] = -gpt2_surprisal_df["surprisal"]
gpt2_surprisal_df["surprisal"] = gpt2_surprisal_df["surprisal"] / np.log(2)
surp_combined_df = pd.concat([bert_surprisal_df, gpt2_surprisal_df], ignore_index=True)
surp_combined_df.to_csv("turkish_bert_gpt2_combined_surprisal.csv", index=False)